In [2]:
import pandas as pd
import psycopg2
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import joblib

# 1. Koneksi ke PostgreSQL
conn = psycopg2.connect(
    dbname='advanced_project',
    user='postgres',
    password='admin123',
    host='localhost',
    port='5432'
)

# 2. Ambil data gabungan
query = """
SELECT 
    s.stu_id,
    s.gender,
    e.course_id,
    e.grade,
    COUNT(sal.activity_id) AS total_activities_completed,
    AVG(EXTRACT(EPOCH FROM (sal.activity_end - sal.activity_start))/60) AS avg_duration_minutes
FROM student s
JOIN enrollment e ON s.stu_id = e.stu_id
JOIN student_activity_log sal ON s.stu_id = sal.stu_id
JOIN course_activity ca ON sal.activity_id = ca.activity_id
WHERE ca.course_id = e.course_id
GROUP BY s.stu_id, s.gender, e.course_id, e.grade
"""

df = pd.read_sql_query(query, conn)
conn.close()

# 3. Cek dan bersihkan missing values
df.dropna(inplace=True)

# 4. Buat label performa berdasarkan grade
def grade_to_performance(grade):
    if grade >= 75:
        return "High"
    elif grade >= 60:
        return "Medium"
    else:
        return "Low"

df["performance"] = df["grade"].apply(grade_to_performance)

# 5. Pastikan semua label tersedia dalam training
expected_genders = ['Male', 'Female']
for g in expected_genders:
    if g not in df["gender"].unique():
        df = pd.concat([df, pd.DataFrame([{
            'stu_id': 'dummy',
            'gender': g,
            'course_id': df["course_id"].iloc[0],
            'grade': 70,
            'total_activities_completed': 0,
            'avg_duration_minutes': 0,
            'performance': 'Medium'
        }])], ignore_index=True)

# 6. Encode categorical features
le_gender = LabelEncoder()
le_course = LabelEncoder()
le_perf = LabelEncoder()

df["gender_encoded"] = le_gender.fit_transform(df["gender"])
df["course_encoded"] = le_course.fit_transform(df["course_id"])
df["performance_encoded"] = le_perf.fit_transform(df["performance"])

# 7. Siapkan fitur dan target
X = df[["gender_encoded", "course_encoded", "total_activities_completed", "avg_duration_minutes"]]
y = df["performance_encoded"]

# 8. Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 9. Latih model
model = RandomForestClassifier(n_estimators=200, max_depth=10, class_weight="balanced", random_state=42)
model.fit(X_train, y_train)

# 10. Evaluasi
y_pred = model.predict(X_test)
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=le_perf.classes_))

# 11. Simpan model & encoders
print("💾 Menyimpan model dan encoder...")
joblib.dump(model, "student_performance_model.pkl")
joblib.dump(le_gender, "label_encoder_gender.pkl")
joblib.dump(le_course, "label_encoder_course.pkl")
joblib.dump(le_perf, "label_encoder_performance.pkl")

print("🎉 Semua file berhasil disimpan!")


C:\Users\ruths\AppData\Local\Temp\ipykernel_11604\3424873489.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


✅ Accuracy: 0.4375
              precision    recall  f1-score   support

        High       0.52      0.73      0.61        49
         Low       0.11      0.06      0.07        18
      Medium       0.28      0.17      0.21        29

    accuracy                           0.44        96
   macro avg       0.30      0.32      0.30        96
weighted avg       0.37      0.44      0.39        96

💾 Menyimpan model dan encoder...
🎉 Semua file berhasil disimpan!
